# Claims — Closed Summary, Open Aging Pareto & Open Tasks (v3 — Commented)
**Generated:** 2025-11-05 20:53:15

In [ ]:
# Parameters
warehouses = ["OKC", "Atlanta", "Orlando", "Ontario", "El Paso", "Flowood", "Phoenix", "Charlotte"]
start_date = None
end_date   = None
max_labels = 25
INPUT_FILES = [("Claims.xlsx", "Claims"), ("Chrysler Claims.xlsx", "Chrysler Claims")]
OUT_XLSX = "claims_analysis_output.xlsx"
FIG_DIR = "figs_claims"

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, PercentFormatter
from pathlib import Path
import re
from typing import Optional, Tuple, Dict
Path(FIG_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# Helpers
def safe_filename(name: str, max_len: int = 180) -> str:
    if name is None:
        name = "untitled"
    name = re.sub(r'[<>:"/\\|?*]', "_", str(name))
    name = re.sub(r"[\x00-\x1f]", "_", name)
    name = re.sub(r"_+", "_", name).strip(" .")
    if not name:
        name = "unnamed"
    return name[:max_len]

def find_column(df: pd.DataFrame, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    return None

def normalize_columns(df: pd.DataFrame):
    wh_col = find_column(df, ["Bucket Name", "Warehouse", "WH", "Facility"])
    sd_col = find_column(df, ["Start Date", "Open Date", "Opened Date", "Created Date", "Create Date", "Claim Open Date", "Date Opened"])
    cd_col = find_column(df, ["Completed Date", "Close Date", "Closed Date"])
    task_col = find_column(df, ["Task Name", "Task"])
    labels_col = find_column(df, ["Labels", "Label"])
    due_col = find_column(df, ["Due Date", "Due date", "Task Due", "Due"])

    if wh_col is None:
        raise KeyError("Warehouse column not found")
    if sd_col is None:
        raise KeyError("Start/Open date column not found")

    out = df.copy()
    out.rename(columns={wh_col: "Warehouse", sd_col: "Start Date"}, inplace=True)
    if cd_col is not None: out.rename(columns={cd_col: "Completed Date"}, inplace=True)
    else: out["Completed Date"] = pd.NaT
    if task_col is not None: out.rename(columns={task_col: "Task Name"}, inplace=True)
    else: out["Task Name"] = ""
    if labels_col is not None: out.rename(columns={labels_col: "Labels"}, inplace=True)
    else: out["Labels"] = ""
    if due_col is not None: out.rename(columns={due_col: "Due Date"}, inplace=True)
    else: out["Due Date"] = pd.NaT

    out["Start Date"] = pd.to_datetime(out["Start Date"], errors="coerce")
    out["Completed Date"] = pd.to_datetime(out["Completed Date"], errors="coerce")
    out["Due Date"] = pd.to_datetime(out["Due Date"], errors="coerce")
    out["Warehouse"] = out["Warehouse"].astype(str).str.strip()
    out["Task Name"] = out["Task Name"].astype(str).str.strip()
    out["Labels"] = out["Labels"].astype(str)
    return out, {}

In [ ]:
def filter_by_warehouses_and_dates(df, warehouses, start_date, end_date):
    wanted = {w.strip().lower(): w for w in warehouses}
    df2 = df[df["Warehouse"].str.lower().isin(wanted.keys())].copy()
    df2["Warehouse"] = df2["Warehouse"].str.lower().map(wanted)
    if start_date:
        df2 = df2[df2["Start Date"] >= pd.to_datetime(start_date)]
    if end_date:
        df2 = df2[df2["Start Date"] <= pd.to_datetime(end_date)]
    return df2

def closed_claims_summary(df, dataset_name):
    closed = df[df["Completed Date"].notna()].copy()
    closed["DaysToClose"] = (closed["Completed Date"] - closed["Start Date"]).dt.days
    agg = closed.groupby("Warehouse", dropna=False)["DaysToClose"].agg(
        CountClosed="count",
        MeanDaysToClose=lambda s: float(np.nanmean(s)) if len(s) else np.nan,
        MedianDaysToClose=lambda s: float(np.nanmedian(s)) if len(s) else np.nan,
        MinDaysToClose=lambda s: float(np.nanmin(s)) if len(s) else np.nan,
        MaxDaysToClose=lambda s: float(np.nanmax(s)) if len(s) else np.nan,
    ).reset_index()
    agg.insert(0, "Dataset", dataset_name)
    return agg

def open_claims_aging_table(df, dataset_name):
    open_df = df[df["Completed Date"].isna()].copy()
    today = pd.Timestamp.today().normalize()
    open_df["AgeDays"] = (today - open_df["Start Date"].dt.normalize()).dt.days
    bucket_labels = ["<30 days", "30-<60 days", "60-<90 days", ">= 90 days"]
    bins = [-1, 29, 59, 89, float("inf")]
    open_df["Aging Bucket"] = pd.cut(open_df["AgeDays"], bins=bins, labels=bucket_labels, include_lowest=True, right=True)
    counts = (open_df.groupby(["Warehouse", "Aging Bucket"]).size().rename("Count").reset_index())
    all_wh = [w for w in warehouses if w in counts["Warehouse"].unique().tolist()]
    full_index = pd.MultiIndex.from_product([all_wh, bucket_labels], names=["Warehouse", "Aging Bucket"])
    counts = counts.set_index(["Warehouse", "Aging Bucket"]).reindex(full_index, fill_value=0).reset_index()
    counts["Percent"] = counts.groupby("Warehouse")["Count"].transform(lambda x: (x / x.sum() * 100.0) if x.sum() else 0.0)
    counts["CumPercent"] = counts.groupby("Warehouse")["Percent"].cumsum()
    counts.insert(0, "Dataset", dataset_name)
    return counts

def plot_pareto_for_wh(df_aging_wh, dataset_name, wh, max_labels=25, save_dir=FIG_DIR):
    labels = df_aging_wh["Aging Bucket"].astype(str).tolist()
    x = np.arange(len(labels))
    counts = df_aging_wh["Count"].to_numpy(dtype=float)
    cum_pct = df_aging_wh["CumPercent"].to_numpy(dtype=float)
    fig, ax = plt.subplots(figsize=(8, 4.6))
    bars = ax.bar(x, counts)
    ax.set_xticks(x); ax.set_xticklabels(labels, rotation=0)
    ax.set_xlabel("Claim Age"); ax.set_ylabel("Open Claims (count)")
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax2 = ax.twinx(); ax2.plot(x, cum_pct, marker="o"); ax2.set_ylabel("Cumulative %")
    ax2.set_ylim(0, 100); ax2.yaxis.set_major_formatter(PercentFormatter(xmax=100))
    total = int(np.nansum(counts))
    ax.set_title(f"{dataset_name} — Open Claims Aging — {wh} (n={total})")
    ax.grid(axis="y", linestyle="--", alpha=0.3); fig.tight_layout()
    if len(labels) <= max_labels:
        for rect, val in zip(bars, counts):
            ax.text(rect.get_x()+rect.get_width()/2, rect.get_height(), f"{int(val)}", ha="center", va="bottom", fontsize=9)
    Path(save_dir).mkdir(parents=True, exist_ok=True)
    fname = f"{safe_filename(dataset_name)}__aging_pareto__{safe_filename(wh)}.png"
    fig.savefig(Path(save_dir)/fname, dpi=150, bbox_inches="tight"); plt.show()

def extract_wh_actions(labels_value: str):
    parts = re.split(r"[;|,]", str(labels_value))
    wh_only = []
    for p in parts:
        s = p.strip()
        if re.match(r"(?i)^WH\b", s):
            wh_only.append(s)
    return wh_only

In [ ]:
# Load/standardize/combine/compute
combined_rows = []
closed_summaries = []
open_aging_tables = []

for rel_path, dname in INPUT_FILES:
    path = Path(rel_path)
    if not path.exists():
        print(f"WARNING: File not found, skipping: {rel_path}")
        continue
    raw = pd.read_excel(path)
    df_std, _ = normalize_columns(raw)
    df_comb = raw.copy()
    for col in ["Warehouse", "Start Date", "Completed Date", "Task Name", "Labels", "Due Date"]:
        if col in df_std.columns:
            df_comb[col] = df_std[col]
    df_comb["Dataset"] = dname
    combined_rows.append(df_comb)
    df_std_f = filter_by_warehouses_and_dates(df_std, warehouses, start_date, end_date)
    closed_summaries.append(closed_claims_summary(df_std_f, dname))
    open_aging_tables.append(open_claims_aging_table(df_std_f, dname))

combined_all = pd.concat(combined_rows, ignore_index=True) if combined_rows else pd.DataFrame()
if not combined_all.empty:
    mask = combined_all["Warehouse"].notna()
    if start_date:
        mask &= pd.to_datetime(combined_all["Start Date"], errors="coerce") >= pd.to_datetime(start_date)
    if end_date:
        mask &= pd.to_datetime(combined_all["Start Date"], errors="coerce") <= pd.to_datetime(end_date)
    wanted = {w.strip().lower() for w in warehouses}
    mask &= combined_all["Warehouse"].astype(str).str.lower().isin(wanted)
    combined_all_filtered = combined_all[mask].copy()
else:
    combined_all_filtered = combined_all

closed_all = pd.concat(closed_summaries, ignore_index=True) if closed_summaries else pd.DataFrame()
aging_all  = pd.concat(open_aging_tables, ignore_index=True) if open_aging_tables else pd.DataFrame()
combined_all_filtered.head(10), closed_all.head(5), aging_all.head(5)

In [ ]:
# Open Tasks – detail & aggregate
open_task_rows = []
for rel_path, dname in INPUT_FILES:
    path = Path(rel_path)
    if not path.exists():
        continue
    raw = pd.read_excel(path)
    df_std, _ = normalize_columns(raw)
    df_std_f = filter_by_warehouses_and_dates(df_std, warehouses, start_date, end_date)
    open_df = df_std_f[df_std_f["Completed Date"].isna()].copy()
    open_df["ActionList"] = open_df["Labels"].apply(extract_wh_actions)
    open_df = open_df[open_df["ActionList"].map(len) > 0].copy().explode("ActionList")
    open_df.rename(columns={"ActionList":"Action Required"}, inplace=True)
    open_df["Dataset"] = dname
    open_task_rows.append(open_df[["Dataset","Warehouse","Task Name","Action Required","Due Date"]])

open_tasks_detail = pd.concat(open_task_rows, ignore_index=True) if open_task_rows else pd.DataFrame(columns=["Dataset","Warehouse","Task Name","Action Required","Due Date"])
if not open_tasks_detail.empty:
    open_tasks_detail = open_tasks_detail.sort_values(["Action Required","Due Date","Warehouse"], na_position="last").reset_index(drop=True)

if not open_tasks_detail.empty:
    open_tasks_agg = (open_tasks_detail.groupby(["Dataset","Warehouse","Action Required"], dropna=False).size().rename("Count").reset_index()
                      .sort_values(["Warehouse","Count"], ascending=[True, False], ignore_index=True))
else:
    open_tasks_agg = pd.DataFrame(columns=["Dataset","Warehouse","Action Required","Count"])

open_tasks_detail.head(10), open_tasks_agg.head(10)

In [ ]:
# Export all outputs to Excel (overwrites file each run to avoid stale data)
with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as writer:
    if not combined_all_filtered.empty:
        combined_all_filtered.to_excel(writer, sheet_name="All_Claims_Combined", index=False)
    if not closed_all.empty:
        closed_all.to_excel(writer, sheet_name="Closed_Claims_Summary", index=False)
    if not aging_all.empty:
        aging_all.to_excel(writer, sheet_name="Open_Claims_Aging", index=False)
    if not open_tasks_detail.empty:
        open_tasks_detail.to_excel(writer, sheet_name="Open_Tasks_By_Warehouse", index=False)
    if not open_tasks_agg.empty:
        open_tasks_agg.to_excel(writer, sheet_name="Open_Tasks_Aggregate", index=False)
print(f"Wrote: {OUT_XLSX}")

In [ ]:
# Pareto chart generation
if not aging_all.empty:
    desired_order = ["<30 days", "30-<60 days", "60-<90 days", ">= 90 days"]
    for dname in aging_all["Dataset"].unique():
        sub = aging_all[aging_all["Dataset"] == dname]
        for wh in warehouses:
            wh_rows = sub[sub["Warehouse"] == wh]
            if wh_rows.empty:
                continue
            wh_rows = wh_rows.set_index("Aging Bucket").reindex(desired_order).reset_index()
            plot_pareto_for_wh(wh_rows, dname, wh, max_labels=max_labels, save_dir=FIG_DIR)
else:
    print("No aging data to plot.")